# Envio de mensagens via WhatsApp Web

Passos:
1) Criar a janela
2) Botão para selecionar arquivo de clientes
3) Label para mostrar caminho do arquivo de clientes
4) Label para mostrar quantidade total de mensagens a serem enviadas (contar quantos telefones E mensagens não vazias existem)
5) Botão para selecionar imagens
6) Label para mostrar caminho das imagens (apenas um unico string com todas as imagens da lista concatenadas)
7) Botão para iniciar processo de envio
8) Processo:  
    8.1) Apenas imprimir as mensagens  
    8.2) Label com dados atualizados na janela:  
        numero de enviados / numero total  
        percentual enviado 

Bibliotecas

In [47]:
# importar as bibliotecas
import tkinter as tk
from tkinter import filedialog as fd
from PIL import Image, ImageTk
import pandas as pd
from pathlib import Path
# importar biblioteca time para criar tempos de espera
import time

import tkinter.scrolledtext as st


In [48]:
# global variables
imgs_path = []
i = 0
j = 0

Funções

In [49]:
def show_imgs():
    
    # função evento mostrar imagem de indice i da lista
    
    global tkfoto

    # parametros canvas
    can_h = 400
    can_w = 400
    
    # esta demorando muito aqui, a solução de usar uma variavel global nao e interessante
    # https://blog.furas.pl/python-tkinter-why-label-doesnt-display-image-bug-with-garbage-collector-in-photoimage-GB.html
    # https://www.c-sharpcorner.com/blogs/basics-for-displaying-image-in-tkinter-python
    
    # obter o elemento i da lista
    foto = Image.open(imgs_path[i])
    
    # obter a tupla com o tamanho
    pic_w, pic_h = foto.size

    # calcular o ratio da imagem (aspecto)
    aspecto = pic_w/pic_h

    # se a imagem é mais larga que alta, a largura limitante sera a maxima do canvas
    if aspecto > 1:
        res_w = can_w
        res_h = can_w / aspecto
    
    # caso contrario, a altura limitante sera a maxima do canvas
    else:
        res_h = can_h
        res_w = can_h * aspecto
    

    # reduzir o tamanho
    foto = foto.resize((int(res_w),int(res_h)))

    # criar o objeto
    tkfoto = ImageTk.PhotoImage(foto)

    # Label para colocar a imagem
    lbl_foto = tk.Label(image=tkfoto,width=can_w,height=can_h,borderwidth=2,relief='solid')
    lbl_foto.grid(row=1,column=4,rowspan=4, padx=10,pady=10)
    
    # mostrar numero da imagem e o nome
    show_name()
    
    return()

In [50]:
def show_name():
    # função para mostrar numero de imagem, total de imagens e nome (sem o caminho)
    # .name extrai o nome do arquivo de um caminho completo
    img_name = '{} de {}: {}'.format(i+1,len(imgs_path),Path(imgs_path[i]).name)
    lbl_imgname = tk.Label(text=img_name,font=('Consolas 10'))
    lbl_imgname.grid(row=5, column=3,columnspan=3,sticky='NSEW',padx=10,pady=10)
    return()
    

In [51]:
def go_rgt():
    # função evento botão para selecionar imagem seguinte (direita)
    global i
    if i < len(imgs_path)-1:
        i = i + 1
    else:
        i = 0
    
    # chamar função para mostrar imagem
    show_imgs()

    return()

In [52]:
def go_lft():
    # função evento botão para selecionar imagem anterior (esquerda)
    global i
    if i > 0:
        i = i - 1
    else:
        i = len(imgs_path)-1
    
    # chamar função para mostrar imagem
    show_imgs()
    
    return()

In [53]:
def sel_imgs():
    # função evento selecionar lista de imagens
        
    global imgs_path
    global i
        
    # selecionar imagens
    imgs_path = fd.askopenfilenames(
        title='Selecione as imagens a enviar',
        filetypes=[
            ('Arquivos de imagen','.jpg'),
            ('Arquivos de imagen','.jpeg'),
            ('Arquivos de imagen','.png'),
            ('Arquivos de imagen','.gif'),]
    )
    
    # Botão para esquerda
    btn_lft = tk.Button(text='<',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_lft)
    btn_lft.grid(row=1,column=3,rowspan=4,sticky='NSEW',padx=10,pady=10)

    # Botão para direita
    btn_rgt = tk.Button(text='>',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_rgt)
    btn_rgt.grid(row=1,column=5,rowspan=4,sticky='NSEW',padx=10,pady=10)
        
    # aqui chamar a função para mostrar a primeira imagem
    i = 0
    show_imgs()

    return()

In [54]:
def sel_file():
    
    # função evento selecionar o arquivo de clientes
    
    global contatos_df

    file_path = fd.askopenfilename(
        title='Selecione o arquivo Excel com a lista de destinatarios',
        filetypes=[
            ('Arquivo Excel','.xls'),
            ('Arquivo Excel','.xlsx')]
        )
    
    # ler arquivo Excel, precisa tres colunas
    contatos_df = pd.read_excel(file_path, sheet_name='CLIENTES')

    # remover registros com mensagens nulas
    contatos_df = contatos_df[~contatos_df['MENSAGEM'].isnull()]

    # resetar o indice
    contatos_df.reset_index(inplace=True)

    # eliminar nova coluna de indice criada
    contatos_df.drop(labels='index',axis=1,inplace=True)

    
    lbl_slctdfile['text'] = 'Serão enviadas {} mensagens.\n Clique em "Enviar Mensagens" para iniciar'.format(contatos_df['MENSAGEM'].count())
    
    return()

In [55]:
def enviar_mensagens():
    lista_enviados = ''
    
    for j, mensagem in enumerate(contatos_df['MENSAGEM']):
        lbl_enviando['text'] = 'Enviando mensagem {} de {} para {}'.format(j,contatos_df['MENSAGEM'].count(),contatos_df['CLIENTE'][j])
        
        # nota: time.sleep não funciona no Tkinter, a menos que seja atualizada a janela
        mainwindow.update()
        time.sleep(1)

        # aqui colocar os resultados de envio.
        # investigar como fazer somente leitura.
        # as configrações de state normal e disabled funcionam mas nao permitem alterar pelo programa        
        # txt_enviados.configure(state ='normal')
        txt_enviados.insert(tk.INSERT,'{} recebeu a mensagem\n'.format(contatos_df['CLIENTE'][j]))
        # txt_enviados.configure(state ='disabled')
        
    return()

Design da janela principal

In [56]:
# criar a janela da aplicação
mainwindow = tk.Tk()

In [57]:
# configurações da janela
# mainwindow.geometry('750x500')
# mainwindow.rowconfigure(0,weight=1)
# mainwindow.columnconfigure([0,1],weight=1)

In [58]:
# Titulo da janela
mainwindow.title("Envio de Mensagens pelo WhatsApp")

''

In [59]:
# Label com titulo principal da janela
lbl_title = tk.Label(text="Envio de Mensagens pelo WhatsApp",font=('Consolas 15 bold underline'),borderwidth=1, relief='solid')
lbl_title.grid(row=0, column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [60]:
# Label de explicação
lbl_desc = tk.Label(text=
    """Este programa envia mensagens pelo WhatsApp Web,
    a partir de uma lista de destinatarios em formato Excel.
    A lista deve conter os campos
    Nome, Telefone e Mensagens, numa folha Clientes"""
    ,font=('Consolas 10'),wraplength=300,borderwidth=1, relief='solid')
lbl_desc.grid(row=1, column=0, columnspan=3,sticky='NSEW',padx=10,pady=10) 

In [61]:
# Label para seleção de arquivo Excel
lbl_file = tk.Label(text='Seleccione o arquivo Excel com os dados:',font=('Consolas 12'),anchor='e')
lbl_file.grid(row=3,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [62]:
# Botao para selecionar o arquivo em Excel
btn_file = tk.Button(text='Clique aqui para seleccionar o arquivo',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_file)
btn_file.grid(row=3,column=2,sticky='NSEW',padx=10,pady=10)

In [63]:
# Caminho e nome do arquivo seleccionado, posteriormente quantidade de mensagens
lbl_slctdfile = tk.Label(text='Nenhum Arquivo Seleccionado',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_slctdfile.grid(row=4,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [64]:
# Label para seleção dos arquivos de imagens
lbl_imgs = tk.Label(text='Seleccione as imagens a enviar:',font=('Consolas 12'),anchor='e')
lbl_imgs.grid(row=2,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [65]:
# Botão para selecionar imagens
btn_imgs = tk.Button(text='Clique aqui para seleccionar as imagens',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_imgs)
btn_imgs.grid(row=2,column=2,sticky='NSEW',padx=10,pady=10)

In [66]:
# Botão para rodar o processo
btn_test = tk.Button(text='Enviar Mensagens',font=('Consolas 10 bold'),command=enviar_mensagens)
btn_test.grid(row=5,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
# pensa que enquanto esta rodando a janela, a informação dentro dela esta disponivel

In [67]:
# Label para colocar informações de estado de mensagen atual
lbl_enviando = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_enviando.grid(row=6,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [68]:
# Scroll text para mostrar a lista
# lbl_enviados = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
# lbl_enviados.grid(row=7,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
txt_enviados = st.ScrolledText(mainwindow,width = 30, 
                            height = 8, 
                            font = ('Consolas 10'))
txt_enviados.grid(row=7,column = 0, columnspan=3,sticky='NSEW', pady = 10, padx = 10)

txt_enviados.insert(tk.INSERT,'')
# investigar como fazer read only


In [69]:
# janela principal
mainwindow.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\pgetar\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\pgetar\AppData\Local\Temp/ipykernel_6076/2452207052.py", line 15, in enviar_mensagens
    txt_enviados.insert(tk.INSERT,'{} recebeu a mensagem\n'.format(contatos_df['CLIENTE'][j]))
  File "c:\Users\pgetar\Anaconda3\lib\tkinter\__init__.py", line 3743, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
_tkinter.TclError: invalid command name ".!frame.!scrolledtext"
